In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import  classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB, GaussianNB
df_train = pd.read_csv('https://raw.githubusercontent.com/ThulaniNyama/south-african-language-identification-hack-2022-data/main/train_set.csv')
print(df_train.head())

c:\Users\F5470242\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\F5470242\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\F5470242\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


  lang_id                                               text
0     xho  umgaqo-siseko wenza amalungiselelo kumaziko ax...
1     xho  i-dha iya kuba nobulumko bokubeka umsebenzi na...
2     eng  the province of kwazulu-natal department of tr...
3     nso  o netefatša gore o ba file dilo ka moka tše le...
4     ven  khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [2]:
df_test =pd.read_csv('https://raw.githubusercontent.com/ThulaniNyama/south-african-language-identification-hack-2022-data/main/test_set.csv')
print(df_test.head())

   index                                               text
0      1  Mmasepala, fa maemo a a kgethegileng a letlele...
1      2  Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2      3         Tshivhumbeo tshi fana na ngano dza vhathu.
3      4  Kube inja nelikati betingevakala kutsi titsini...
4      5                      Winste op buitelandse valuta.


In [3]:
df_train.isnull().sum()

lang_id    0
text       0
dtype: int64

In [4]:
df_train["lang_id"].value_counts()

xho    3000
eng    3000
nso    3000
ven    3000
tsn    3000
nbl    3000
zul    3000
ssw    3000
tso    3000
sot    3000
afr    3000
Name: lang_id, dtype: int64

In [5]:
x = df_train["text"]
y = df_train["lang_id"]

# cv = TfidfVectorizer(ngram_range=(6,6), analyzer='char', min_df=2, max_df=0.9, stop_words='english')
cv = CountVectorizer(ngram_range=(6,6), analyzer='char', min_df=2, max_df=0.9, stop_words='english')
X = cv.fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.01, 
                                                    random_state=10)

In [6]:
print(X_train.shape)

(32670, 422254)


In [7]:
tester = cv.transform(df_test["text"])

In [8]:
MNBC2 = MultinomialNB(alpha=0.02)
MNBC2.fit(X_train, y_train)
MNBC2.score(X_test, y_test)
y_pred = MNBC2.predict(tester)

In [9]:
MNBC = MultinomialNB(alpha=0.01)
MNBC.fit(X_train, y_train)
MNBC.score(X_test, y_test)
y_pred = MNBC.predict(tester)

In [10]:
CNBC = ComplementNB(alpha=0.01)
CNBC.fit(X_train, y_train)
CNBC.score(X_test, y_test)
y_pred = CNBC.predict(tester)

In [11]:
# from sklearn.linear_model import RidgeClassifier
# RC = RidgeClassifier(alpha = 0.1)
# RC.fit(X_train, y_train)
# RC.score(X_test, y_test)
# y_pred = RC.predict(tester)

In [12]:
# from sklearn.naive_bayes import BernoulliNB
# BNBC = BernoulliNB(alpha=0.2)
# BNBC.fit(X_train, y_train)
# BNBC.score(X_test, y_test)
# y_pred = BNBC.predict(tester)

In [13]:
# print(BNBC.score(X_test, y_test))
print(CNBC.score(X_test, y_test))
print(MNBC.score(X_test, y_test))
# print(RC.score(X_test, y_test))
print(MNBC2.score(X_test, y_test))

1.0
1.0
1.0


In [14]:
# from sklearn.ensemble import VotingClassifier
# vc = VotingClassifier(estimators=[
#         ('BNBC', BNBC),
#         ('CNBC', CNBC),
#     ],
#     voting='soft',
#     n_jobs=-1
# )


In [15]:
# vc.fit(X_train, y_train)
# vc.score(X_test, y_test)
# y_pred = vc.predict(tester)

In [16]:
from sklearn.ensemble import StackingClassifier
sc = StackingClassifier(
    estimators=[
        ('CNBC', CNBC),
        ('MNBC2', MNBC2)
    ],
    final_estimator=MNBC2,
    n_jobs=-1,
    passthrough=True
)

In [17]:
sc.fit(X_train, y_train)
sc.score(X_test, y_test)
y_pred = sc.predict(tester)

In [18]:
print(sc.score(X_test, y_test))

1.0


In [19]:
from sklearn.preprocessing import StandardScaler

In [20]:
from sklearn.ensemble import StackingClassifier
sc2 = StackingClassifier(
    estimators=[
        ('CNBC', sc),
        ('MNBC', MNBC)
    ],
    final_estimator=sc,
    n_jobs=-1,
    passthrough=True
)

In [21]:
sc2.fit(X_train, y_train)
sc2.score(X_test, y_test)
y_pred = sc.predict(tester)

In [22]:
# print(vc.score(X_test, y_test))

In [23]:
print(sc2.score(X_test, y_test))

1.0


In [24]:
# print(classification_report(y_test, y_pred))

In [25]:
submission_df = pd.DataFrame(df_test['index'])
submission_df['lang_id'] = y_pred
submission_df.to_csv('Final.csv', index=False)